In [ ]:
# default_exp experiment.lake_experiment

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import json

# export
import os
import pickle
import tempfile
import uuid
import warnings
from copy import copy
from pathlib import Path
from typing import Dict, Set

import boto3
import pandas as pd
from IPython import display
from IPython.display import HTML, IFrame
from pyrsistent import freeze, thaw

from sciflow.s3_utils import S3File, load_json, put_data, s3_join
from sciflow.utils import prepare_env

In [ ]:
# export
file_to_mime_type_map = {
    ".txt": "text/csv",
    ".csv": "text/csv",
    ".png": "image/png",
    ".jpg": "image/jpeg",
    ".mp4": "video/mp4",
    ".pickle": "application/octet-stream",
}

In [ ]:
# export


class Artifact:
    """Displays or saves an artifact."""

    can_render: Set[str] = set()

    def __init__(self, name: str, file, content_type: str = None):
        self.name = name
        self.file = file
        self.content_type = content_type
        self.extension = (
            "" if self.content_type is None else self.content_type.split("/")[-1]
        )
        self._content = None
        self._rendered = None

    def __repr__(self):
        return f"{self.__class__.__name__}(name={self.name})"

    def render(self):
        """Render the artifact according to its content-type."""
        if self._rendered is None:
            self._rendered = self._render()
        return self._rendered

    def _render(self):
        """Return the object that represents the rendered artifact."""
        raise NotImplementedError

    def show(self):
        warnings.warn(
            "`show` is deprecated in favor of `render` and will removed in a future release.",
            DeprecationWarning,
            stacklevel=2,
        )
        return self.render()

    def save(self, to_dir: str = "") -> None:
        """
        Save artifact to disk.

        Args:
            to_dir: Directory in which to save the artifact. Defaults to the current working directory.

        """
        if to_dir:
            os.makedirs(str(to_dir), exist_ok=True)
        with open(os.path.join(str(to_dir), self._make_filename()), "wb") as file:
            file.write(self.content)

    def as_content_type(self, content_type: str) -> "Artifact":
        """Interpret artifact as being of content-type."""
        try:
            artifact_type = content_type_to_artifact_cls[content_type]
        except KeyError:
            raise ValueError(
                f"Incense does not have a class that maps to content-type {content_type}"
            )
        else:
            return self.as_type(artifact_type)

    def as_type(self, artifact_type) -> "Artifact":
        self.file.seek(0)
        return artifact_type(self.name, self.file)

    @property
    def content(self):
        """Access the raw bytes of the artifact."""
        if self._content is None:
            self._content = self.file.read()
        return self._content

    def _make_filename(self):
        # TODO does this work on gridfs file?
        exp_id, artifact_name = self.file.name.split("/")[-2:]
        return f"{exp_id}_{artifact_name}" + (
            "" if artifact_name.endswith(self.extension) else f".{self.extension}"
        )


class ImageArtifact(Artifact):
    """Displays or saves an image artifact."""

    can_render = {"image/png", "image/jpeg"}

    def _render(self):
        return display.Image(data=self.content)


class MP4Artifact(Artifact):
    """Displays or saves a MP4 artifact"""

    can_render = {"video/mp4"}

    def _render(self):
        self.save()
        return HTML(
            f"""
        <video width="640" height="480" controls autoplay>
          <source src="{self._make_filename()}" type="video/mp4">
        </video>
        """
        )


class CSVArtifact(Artifact):
    """Displays and saves a CSV artifact"""

    can_render = {"text/csv"}

    def _render(self):
        return pd.read_csv(self.file)


class PickleArtifact(Artifact):
    """Displays and saves a Pickle artifact"""

    can_render: Set[str] = set()

    def __init__(self, name: str, file, content_type: str = None):
        super().__init__(name, file, content_type)
        self.extension = "pickle"

    def _render(self):
        return pickle.load(self.file)


class PDFArtifact(Artifact):
    """Displays and saves a PDF artifacts."""

    can_render = {"application/pdf"}

    # TODO probably needs jupyter extension to be able to display pdf.
    def _render(self):
        return IFrame(self._make_filename(), width=600, height=300)


content_type_to_artifact_cls = {}
for cls in copy(locals()).values():
    if isinstance(cls, type) and issubclass(cls, Artifact):
        for content_type in cls.can_render:
            content_type_to_artifact_cls[content_type] = cls

In [ ]:
# export


class LakeExperiment:
    def __init__(
        self, bucket_name, base_key, experiment_id, start_time, data, name=None
    ):
        self.bucket_name = bucket_name
        self.base_key = base_key
        self.experiment_id = experiment_id
        self.metrics_key = s3_join(self.base_key, "metrics", experiment_id)
        self.artifacts_key = s3_join(self.base_key, "artifacts", experiment_id)
        self.start_time = start_time
        self._data = freeze(data)
        # TODO is a name needed?
        # self.name = self.experiment_id if name is None else name
        self.name = name
        self.s3_res = boto3.resource("s3")
        self.bucket = self.s3_res.Bucket(self.bucket_name)
        self.artifacts = self._load_artifacts()
        self.metrics = self._load_metrics()

    def __repr__(self):
        if self.name is None:
            text = f"{self.__class__.__name__}(id={self.experiment_id}, start_time={datetime.datetime.fromtimestamp(round(self.start_time))})"
        else:
            text = f"{self.__class__.__name__}(id={self.experiment_id}, name={self.name} start_time={datetime.datetime.fromtimestamp(round(self.start_time))})"
        return text

    def __getattr__(self, item):
        """Try to relay attribute access to easy dict, to allow dotted access."""
        return getattr(self._data, item)

    def to_dict(self) -> dict:
        return thaw(self._data)

    def _load_artifacts(self) -> Dict[str, Artifact]:
        artifacts = {}

        artifact_keys = [
            obj.key for obj in self.bucket.objects.filter(Prefix=self.artifacts_key)
        ]

        for artifact_key in artifact_keys:
            s3_object = self.s3_res.Object(
                bucket_name=self.bucket_name, key=artifact_key
            )
            artifact_file = S3File(s3_object)
            name = os.path.basename(artifact_key)

            try:
                content_type = file_to_mime_type_map[os.path.splitext(name)[-1]]
                artifact_type = content_type_to_artifact_cls[content_type]
                artifacts[name] = artifact_type(
                    name, artifact_file, content_type=content_type
                )
            except KeyError:
                artifact_type = Artifact
                artifacts[name] = artifact_type(name, artifact_file)

        return artifacts

    def _load_metrics(self) -> Dict[str, pd.Series]:
        metrics = {}
        metric_keys = self.bucket.objects.filter(Prefix=self.metrics_key)
        for metric_key in metric_keys:
            metrics_entry = load_json(self.s3_res, self.bucket_name, metric_key.key)
            for metric_name in metrics_entry.keys():
                metrics[metric_name] = pd.Series(
                    data=metrics_entry[metric_name]["values"],
                    index=pd.Index(metrics_entry[metric_name]["steps"], name="step"),
                    name=metric_name,
                )
        return metrics

    def metrics_as_df(self):
        return pd.concat(self.metrics.values(), axis=1)

    def delete(self, confirmed: bool = False):
        raise NotImplementedError

    def _delete(self):
        raise NotImplementedError

    def _delete_metrics(self):
        raise NotImplementedError

    def _delete_artifacts(self):
        raise NotImplementedError

In [ ]:
prepare_env()
_bucket_name = os.environ["SCIFLOW_BUCKET"]

In [ ]:
today = datetime.datetime.utcnow().strftime("%Y%m%d")
_base_key = f"sciflow-experiment-testing-{today}"
_run_id = f"experiment_{str(uuid.uuid4())[-6:]}"
_run_key = s3_join(_base_key, _run_id)
_metrics_key = f"{_base_key}/experiments/metrics"
_artifacts_key = f"{_base_key}/experiments/artifacts"
_runs_key = f"{_base_key}/experiments/runs"
_s3_res = boto3.resource("s3")
_s3_client = boto3.client("s3")
_bucket = _s3_res.Bucket(_bucket_name)

In [ ]:
%matplotlib auto

# Set-up - Create Test Data

In [ ]:
# export


def create_experiment_test_data(
    s3_res, s3_client, bucket_name, base_key, experiment_id
):
    metrics_key = f"{base_key}/experiments/metrics"
    artifacts_key = f"{base_key}/experiments/artifacts"
    runs_key = f"{base_key}/experiments/runs"

    df = pd.DataFrame({"a": [1, 2, 3], "b": ["a", "b", "c"]})

    with tempfile.TemporaryDirectory() as temp_dir:
        csv_path = f"{temp_dir}/testfile.csv"
        df.to_csv(csv_path)
        fig = df.a.plot.hist().figure
        pdf_path = f"{temp_dir}/testfile.pdf"
        fig.savefig(pdf_path)
        pickle_path = f"{temp_dir}/testfile.pkl"
        df.to_pickle(pickle_path)
        for artifact_path in (csv_path, pdf_path, pickle_path):
            s3_client.upload_file(
                artifact_path,
                bucket_name,
                s3_join(artifacts_key, experiment_id, Path(artifact_path).name),
            )

    metrics = {
        "f1": {
            "steps": [
                0,
                1,
            ],
            "timestamps": [
                "2022-06-09T19:02:33.778171",
                "2022-06-09T19:02:34.014545",
            ],
            "values": [0.37, 0.45],
        },
        "precision": {
            "steps": [
                0,
                1,
                2,
            ],
            "timestamps": [
                "2022-06-09T19:02:33.778171",
                "2022-06-09T19:02:34.014545",
                "2022-06-09T19:02:34.014545",
            ],
            "values": [0.37, 0.45, 0.89],
        },
    }

    put_data(
        s3_res,
        bucket_name,
        s3_join(metrics_key, experiment_id, "metrics.json"),
        json.dumps(metrics),
    )

    experiment_data = {
        "meta": "{ }",
        "steps": '[ "experiment-test" ]',
        "start_time": 1654801353.1519804,
        "heartbeat": None,
        "status": "COMPLETED",
        "host": '{\n  "ENV" : { },\n  "cpu" : "777 CPU @ 9.99GHz",\n  "hostname" : "sciflow",\n  "os" : [ "Linux", "Linux-5.555-555" ],\n  "python_version" : "3.9.0"\n}',
        "dir0": "sample_flow_instance_3eadd8",
        "stop_time": 1654801361.0863302,
        "captured_out": "******BEGIN step: experiment-test******\n******END step: experiment-test******\n",
        "resources": "[ ]",
        "artifacts": '[ "testfile.csv", "testfile.pdf", "testfile.pkl" ]',
        "elapsed_time": 7.93,
        "info": "{ }",
        "all_hosts": '{\n  "experiment-test" : {\n    "ENV" : { },\n    "cpu" : "777 CPU @ 9.99GHz",\n    "hostname" : "sciflow",\n    "os" : [ "Linux", "Linux-5.555-555" ],\n    "python_version" : "3.9.0"\n  }\n}',
        "experiment": "{ }",
        "experiment_id": "sample_flow_instance_3eadd8",
    }

    put_data(
        s3_res,
        bucket_name,
        s3_join(runs_key, experiment_id, "run.json"),
        json.dumps(experiment_data),
    )

    return experiment_data

In [ ]:
experiment_data = create_experiment_test_data(
    _s3_res, _s3_client, _bucket_name, _base_key, _run_id
)

In [ ]:
import time

experiment = LakeExperiment(
    _bucket_name, _base_key, _run_id, time.time(), experiment_data, name="quick-test"
)

In [ ]:
experiment._load_metrics()

In [ ]:
experiment = LakeExperiment(
    _bucket_name, _base_key, _run_id, time.time(), experiment_data
)
experiment

In [ ]:
experiment._load_metrics()